<a href="https://colab.research.google.com/github/konstantin-larin/football-predictor/blob/main/Football_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install kagglehub
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.6 MB/s eta 0:00:00


In [38]:
#Load datasets from https://www.kaggle.com/datasets/davidcariboo/player-scores/

import kagglehub

# Download latest version
path = kagglehub.dataset_download("davidcariboo/player-scores")

# appearances = pd.read_csv(path + '/appearances.csv')
# club_games = pd.read_csv(path + '/club_games.csv')
clubs = pd.read_csv(path + '/clubs.csv')
competitions = pd.read_csv(path + '/competitions.csv') #applied
game_events = pd.read_csv(path + '/game_events.csv')
game_lineups = pd.read_csv(path + '/game_lineups.csv')
games = pd.read_csv(path + '/games.csv')
players = pd.read_csv(path + '/players.csv')
player_valuations = pd.read_csv(path + '/player_valuations.csv')
transfers = pd.read_csv(path + '/transfers.csv')

# add elo_ratings
add_path = kagglehub.dataset_download("adamgbor/club-football-match-data-2000-2025")
elo_ratings = pd.read_csv(add_path + '/EloRatings.csv')

Подготавливаем elo_ratings для присоединения к clubs

In [39]:
clubs = clubs.dropna(subset=['club_code'])

elo_ratings_copy = elo_ratings.copy().groupby('club', as_index=False).agg({'elo': 'mean'}).rename(columns={'club': 'club_code'})
elo_ratings_copy['elo'] = elo_ratings['elo'].astype('int')
def club_name_to_code(x):
  x = x.lower()
  words = [word for word in list(x.split(' ')) if '.' not in word]
  return "-".join(words)

elo_ratings_copy['club_code'] = elo_ratings_copy['club_code'].apply(club_name_to_code)
elo_clubs = set(elo_ratings_copy['club_code'])
min_elo = min(elo_ratings_copy['elo'].to_numpy().reshape(-1))

def find_elo(cur_code):
    for club_code in elo_clubs:
      if club_code in cur_code:
        # elo_clubs.remove(club_code)
        return elo_ratings_copy.loc[elo_ratings_copy['club_code'] == club_code, 'elo'].values[0]
    return min_elo

clubs['elo'] = clubs['club_code'].apply(find_elo)

View the data

In [87]:
# games.head(
# competitions.head()
# clubs['club_code'].unique()
# club_games.head()
# games.columns
# game_events.head()
# game_lineups.head()
# player_valuations.head()
# games['round'].unique()
# np.sort(clubs['name'])
# appearances.head()
# games[games['home_club_position'].isna()]
# players.head()
# player_valuations[player_valuations['player_id'] == 405973]
# transfers.head()
# games['round'].unique()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
1555,2320460,DFB,2013,First Round,2013-08-04,1.0,86.0,3.0,1.0,NaN,...,Ludwigsparkstadion,19000.0,Robert Kempter,https://www.transfermarkt.co.uk/spielbericht/i...,4-2-3-1,4-3-1-2,NaN,Sportverein Werder Bremen von 1899,3:1,domestic_cup
8977,4353903,DFB,2024,First Round,2024-08-18,1.0,4.0,4.0,6.0,NaN,...,Ludwigsparkstadion,14868.0,Bastian Dankert,https://www.transfermarkt.co.uk/spielbericht/i...,5-4-1,4-2-3-1,NaN,1.FC Nuremberg,4:6,domestic_cup
9014,2370752,DFB,2013,Round of 16,2013-12-03,1.0,16.0,0.0,2.0,NaN,...,Ludwigsparkstadion,30931.0,Marco Fritz,https://www.transfermarkt.co.uk/spielbericht/i...,4-2-3-1,4-2-3-1,NaN,Borussia Dortmund,0:2,domestic_cup
10939,3245513,DFB,2019,Second Round,2019-10-29,1.0,3.0,3.0,2.0,NaN,...,Hermann-Neuberger-Stadion,6800.0,Martin Petersen,https://www.transfermarkt.co.uk/spielbericht/i...,3-5-2,4-4-2 double 6,NaN,1.FC Köln,3:2,domestic_cup
12748,4240125,DFB,2023,Quarter-Finals,2024-03-12,1.0,18.0,2.0,1.0,NaN,...,Ludwigsparkstadion,15903.0,Robert Hartmann,https://www.transfermarkt.co.uk/spielbericht/i...,3-5-2 flat,3-4-2-1,NaN,Borussia Verein für Leibesübungen 1900 Mönchen...,2:1,domestic_cup
16497,4222798,DFB,2023,Round of 16,2023-12-06,1.0,24.0,2.0,0.0,NaN,...,Ludwigsparkstadion,15903.0,Daniel Siebert,https://www.transfermarkt.co.uk/spielbericht/i...,5-4-1,4-1-4-1,NaN,Eintracht Frankfurt Fußball AG,2:0,domestic_cup
18637,3195542,DFB,2019,First Round,2019-08-11,1.0,109.0,3.0,2.0,NaN,...,Hermann-Neuberger-Stadion,5021.0,Sven Waschitzki-Günther,https://www.transfermarkt.co.uk/spielbericht/i...,3-5-2,4-4-2 double 6,NaN,NaN,3:2,domestic_cup
26796,3332669,DFB,2019,Semi-Finals,2020-06-09,1.0,15.0,0.0,3.0,NaN,...,Hermann-Neuberger-Stadion,NaN,Guido Winkmann,https://www.transfermarkt.co.uk/spielbericht/i...,5-4-1,4-2-3-1,NaN,Bayer 04 Leverkusen Fußball,0:3,domestic_cup
27896,2864716,DFB,2017,First Round,2017-08-13,1.0,89.0,1.0,2.0,NaN,...,Hermann-Neuberger-Stadion,6936.0,Sören Storks,https://www.transfermarkt.co.uk/spielbericht/i...,3-5-2,4-3-3 Defending,NaN,1. Fußballclub Union Berlin,1:2,domestic_cup
46708,3319622,DFB,2019,Quarter-Finals,2020-03-03,1.0,38.0,8.0,7.0,NaN,...,Hermann-Neuberger-Stadion,6800.0,Deniz Aytekin,https://www.transfermarkt.co.uk/spielbericht/i...,4-4-2 double 6,3-4-1-2,NaN,Fortuna Düsseldorf,8:7,domestic_cup


In [137]:
import datetime
target_dict = {
    0: 'Ничья',
    1: 'Победа домашней команды',
    2: 'Победа гостевой команды',
}



data = games.copy().drop(
      ['url', 'aggregate', 'home_club_name', 'away_club_name', 'game_id', 'competition_type', #точно нет
        'season', #здесь еще date
        'stadium', 'referee', 'attendance',
       'home_club_manager_name', 'away_club_manager_name', 'home_club_formation', 'away_club_formation', #также в целом состав игроков
       'competition_id'
       ],
      axis=1).dropna(subset=['home_club_id', 'away_club_id'])

  # убираем ненужные игры
data = data[data['round'] != '|']
# убираем игры, где недостаточно информации о клубах
data = data[(data['home_club_id'].isin(clubs['club_id'])) & (data['away_club_id'].isin(clubs['club_id']))]
  # считаем целевую переменную
target = np.where(data['home_club_goals'] > data['away_club_goals'], 1, np.where(data['home_club_goals'] == data['away_club_goals'], 0, 2))
data.drop(columns=['home_club_goals', 'away_club_goals'], inplace=True)


data['is_matchday'] = np.where(data['round'].str.contains('atchday'), 1, 0)
data.drop(columns=['round'], inplace=True)




# валидационная выборка будет состоять из матчей, чья давность не более 30 дней относительно текущего момента
current_datetime = datetime.datetime.today()
days_ago = (current_datetime - pd.to_datetime(data['date'])).dt.days
data.drop(columns=['date'], inplace=True)  #убираем столбец
is_recent_game = days_ago <= 30

X_val = data[is_recent_game]
data = data[~is_recent_game]
y_val = target[is_recent_game]
target = target[~is_recent_game]
data.head()

,home_club_id,away_club_id,home_club_position,away_club_position,is_matchday
0,33.0,41.0,8.0,9.0,1
1,23.0,86.0,13.0,7.0,1
2,16.0,23.0,1.0,15.0,1
3,23.0,24.0,18.0,11.0,1
4,16.0,41.0,1.0,15.0,1


Преобразовываем признаки для входа в модель

In [138]:
from sklearn.preprocessing import LabelEncoder
from catboost import Pool
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

#общее
categorical_features = ['home_club_id', 'away_club_id', 'is_matchday']

numerical_features = []
data['home_club_id'] = data['home_club_id'].astype('int64')
data['away_club_id'] = data['away_club_id'].astype('int64')
X_val['home_club_id'] = X_val['home_club_id'].astype('int64')
X_val['away_club_id'] = X_val['away_club_id'].astype('int64')

#разбиение данных, балансировка классов
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)
class_weights = class_weight.compute_class_weight('balanced', classes=np.array(list(target_dict.keys())), y=y_train)
class_weights_dict = dict(zip(list(target_dict.keys()), class_weights))

#трансформация для Random Forest
rfc_X_train = X_train.copy()
rfc_X_test = X_test.copy()
rfc_X_val = X_val.copy()

# для random forest
max_position = max(max(data['home_club_position'].to_numpy().reshape(-1)), max(data['away_club_position'].to_numpy().reshape(-1)))

#те клубы, у которых остались пропуски в позициях играли только в каких-то кубковых матчах, то есть их игр зафиксировано мало, а значит такие клубы
#не являются популярными
home_mean_positions = data.groupby('home_club_id', as_index=False).agg({'home_club_position': 'mean'}).fillna(max_position) \
                      .rename(columns={'home_club_id': 'id', 'home_club_position': 'pos'})
home_mean_positions['pos'] = home_mean_positions['pos'].astype('float')
away_mean_positions = data.groupby('away_club_id', as_index=False).agg({'away_club_position': 'mean'}).fillna(max_position).rename(columns={'away_club_id': 'id', 'away_club_position': 'pos'})
away_mean_positions['pos'] = away_mean_positions['pos'].astype('float')
mean_positions = pd.merge(
    home_mean_positions,
    away_mean_positions,
    on='id',
    how='outer',
    suffixes=('_home', '_away')
)
def calc_mean_position(row):
  home = row['pos_home']
  away = row['pos_away']
  if pd.isna(home):
    return away
  if pd.isna(away):
    return home
  return np.round((home + away) / 2)
mean_positions['pos'] = mean_positions.apply(calc_mean_position, axis=1)

def apply_mean_position(row):
  home_id = row['home_club_id']
  away_id = row['away_club_id']
  if pd.isna(row['home_club_position']):
    row['home_club_position'] = mean_positions.loc[mean_positions['id'] == home_id, 'pos'].values[0]
  if pd.isna(row['away_club_position']):
    row['away_club_position'] = mean_positions.loc[mean_positions['id'] == away_id, 'pos'].values[0]

# rfc_X_train = rfc_X_train.apply(apply_mean_position, axis=1)
print(missing_home_ids)
# rfc_X_test = rfc_X_test.apply(apply_mean_position, axis=1)
# rfc_X_val = rfc_X_val.apply(apply_mean_position, axis=1)

# rfc_X_train['home_club_position'] = rfc_X_train.apply(apply_mean_position)
# rfc_X_train['away_club_position'] = rfc_X_train['away_club_position'].fillna(0)
# rfc_X_test['home_club_position'] = rfc_X_test['home_club_position'].fillna(0)
# rfc_X_test['away_club_position'] = rfc_X_test['away_club_position'].fillna(0)
# rfc_X_val['home_club_position'] = rfc_X_val['home_club_position'].fillna(0)
# rfc_X_val['away_club_position'] = rfc_X_val['away_club_position'].fillna(0)

print(rfc_X_train.head())
#трансформация для CatBoost
categorical_features_indices = [data.columns.get_loc(col) for col in categorical_features]
rfc_X_train, rfc_X_test, rfc_X_val = rfc_X_train.to_numpy(), rfc_X_test.to_numpy(), rfc_X_val.to_numpy()
# X_train.head()

set()
       home_club_id  away_club_id  home_club_position  away_club_position  \
5971          18303         18105                 3.0                 8.0   
23543           273           368                 NaN                 NaN   
56013           370           987                 2.0                10.0   
53750           660          6992                 1.0                 6.0   
25799           172           475                 3.0                 9.0   

       is_matchday  
5971             1  
23543            0  
56013            1  
53750            1  
25799            1  


напишем универсальную функцию для теста модели

In [139]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def test_model(model, X_test, y_test):
  y_pred_proba = np.clip(model.predict_proba(X_test), 1e-15, 1)
  y_pred = np.argmax(y_pred_proba, axis=1)
  #Используем метрики для оценки точности модели
  accuracy = accuracy_score(y_test, y_pred)
  print('Общая доля верно классифицированных исходов матчей:', np.round(accuracy, 2))

  precision_draw = precision_score(y_test == 0, y_pred == 0)
  print('Доля правильно предсказанных ничей:', np.round(precision_draw, 2))
  precision_1 = precision_score(y_test == 1, y_pred == 1)
  print('Доля правильно предсказанных побед домашней команды:', np.round(precision_1, 2))
  precision_2 = precision_score(y_test == 2, y_pred == 2)
  print('Доля правильно предсказанных побед гостевой команды:', np.round(precision_2, 2))

  recall_draw = recall_score(y_test == 0, y_pred == 0)
  print('Доля фактически наступивших ничей, которые были верно предсказаны:', np.round(recall_draw, 2))
  recall_1 = recall_score(y_test == 1, y_pred == 1)
  print('Доля фактически наступивших побед домашней команды, которые были верно предсказаны:', np.round(recall_1, 2))
  recall_2 = recall_score(y_test == 2, y_pred == 2)
  print('Доля фактически наступивших побед гостевой команды, которые были верно предсказаны:', np.round(recall_2, 2))

  f1_draw = f1_score(y_test == 0, y_pred == 0)
  print('F1-мера для ничей:', np.round(f1_draw, 2))
  f1_1 = f1_score(y_test == 1, y_pred == 1)
  print('F1-мера для побед домашней команды:', np.round(f1_1, 2))
  f1_2 = f1_score(y_test == 2, y_pred == 2)
  print('F1-мера для побед гостевой команды:', np.round(f1_2, 2))

  f1_average = (f1_1 + f1_2 + f1_draw) / 3
  print("Средняя F1-мера:", np.round(f1_average, 2))

  max_proba = np.max(y_pred_proba, axis=1)
  print("Средняя максимальная вероятность:", np.round(np.mean(max_proba), 2))
  sorted_proba = np.sort(y_pred_proba, axis=1)[:, ::-1]
  confidences = sorted_proba[:, 0] - sorted_proba[:, 1]
  print('Средняя уверенность в предсказании:', np.round(np.mean(confidences), 2)) # от 0 - абсолютно уверенная модель, 1 - очень неуверенная

Обучаем RandomForestClassifier.

In [140]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(criterion='gini', bootstrap=True, random_state=42, class_weight=class_weights_dict)
rfc.fit(rfc_X_train, y_train)
feature_importances = rfc.feature_importances_
print('Важность признаков для модели:')
for i, feature in enumerate(data.columns):
  print(f"{feature}: {np.round(feature_importances[i], 2)}")

Важность признаков для модели:
home_club_id: 0.36
away_club_id: 0.35
home_club_position: 0.15
away_club_position: 0.14
is_matchday: 0.0


Смотрим результаты RandomForestClassifier на тестовой выборке

In [141]:
test_model(rfc, rfc_X_test, y_test)

Общая доля верно классифицированных исходов матчей: 0.53
Доля правильно предсказанных ничей: 0.31
Доля правильно предсказанных побед домашней команды: 0.61
Доля правильно предсказанных побед гостевой команды: 0.52
Доля фактически наступивших ничей, которые были верно предсказаны: 0.22
Доля фактически наступивших побед домашней команды, которые были верно предсказаны: 0.69
Доля фактически наступивших побед гостевой команды, которые были верно предсказаны: 0.55
F1-мера для ничей: 0.26
F1-мера для побед домашней команды: 0.65
F1-мера для побед гостевой команды: 0.54
Средняя F1-мера: 0.48
Средняя максимальная вероятность: 0.66
Средняя уверенность в предсказании: 0.41


Смотрим результаты RandomForestClassifier на валидационной выборке (актуальные матчи)

In [142]:
test_model(rfc, rfc_X_val, y_val)

Общая доля верно классифицированных исходов матчей: 0.54
Доля правильно предсказанных ничей: 0.33
Доля правильно предсказанных побед домашней команды: 0.61
Доля правильно предсказанных побед гостевой команды: 0.56
Доля фактически наступивших ничей, которые были верно предсказаны: 0.24
Доля фактически наступивших побед домашней команды, которые были верно предсказаны: 0.68
Доля фактически наступивших побед гостевой команды, которые были верно предсказаны: 0.58
F1-мера для ничей: 0.28
F1-мера для побед домашней команды: 0.64
F1-мера для побед гостевой команды: 0.57
Средняя F1-мера: 0.5
Средняя максимальная вероятность: 0.63
Средняя уверенность в предсказании: 0.37


Обучаем CatBoostClassifier. Оцениваем важность признаков

In [149]:
from catboost import CatBoostClassifier

cbc = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=7,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=False,
    cat_features=categorical_features_indices,
    early_stopping_rounds=50,
    class_weights=class_weights_dict
)

cbc.fit(X_train, y_train, eval_set=(X_test, y_test))

feature_importances = cbc.feature_importances_
print('Важность признаков для модели:')
for i, feature in enumerate(X_train.columns):
  print(f"{feature}: {np.round(feature_importances[i] / 100, 2)}")

Важность признаков для модели:
home_club_id: 0.11
away_club_id: 0.1
home_club_position: 0.39
away_club_position: 0.4
is_matchday: 0.0


Смотрим результаты CatBoost на тестовой выборке

In [150]:
test_model(cbc, X_test, y_test)

Общая доля верно классифицированных исходов матчей: 0.57
Доля правильно предсказанных ничей: 0.35
Доля правильно предсказанных побед домашней команды: 0.71
Доля правильно предсказанных побед гостевой команды: 0.58
Доля фактически наступивших ничей, которые были верно предсказаны: 0.4
Доля фактически наступивших побед домашней команды, которые были верно предсказаны: 0.62
Доля фактически наступивших побед гостевой команды, которые были верно предсказаны: 0.63
F1-мера для ничей: 0.37
F1-мера для побед домашней команды: 0.66
F1-мера для побед гостевой команды: 0.6
Средняя F1-мера: 0.55
Средняя максимальная вероятность: 0.53
Средняя уверенность в предсказании: 0.22


Смотрим результаты CatBoost на валидационной выборке (актуальные матчи)

In [151]:
test_model(cbc, X_val, y_val)

Общая доля верно классифицированных исходов матчей: 0.5
Доля правильно предсказанных ничей: 0.26
Доля правильно предсказанных побед домашней команды: 0.65
Доля правильно предсказанных побед гостевой команды: 0.55
Доля фактически наступивших ничей, которые были верно предсказаны: 0.34
Доля фактически наступивших побед домашней команды, которые были верно предсказаны: 0.56
Доля фактически наступивших побед гостевой команды, которые были верно предсказаны: 0.53
F1-мера для ничей: 0.3
F1-мера для побед домашней команды: 0.6
F1-мера для побед гостевой команды: 0.54
Средняя F1-мера: 0.48
Средняя максимальная вероятность: 0.53
Средняя уверенность в предсказании: 0.21


Использование модели.
На вход подаются коды клубов (сделано для удобства)
Из уже доступных датасетов или стороних ресурсов составляется объект.
Модель делает предикт для этого объекта.

In [71]:
club_codes = dict(zip(clubs['club_code'], clubs['club_id']))

def predict_match(home_code, away_code):
  home_id = club_codes[home_code]
  away_id = club_codes[away_code]
  print(home_id)
  # x = [[home_id, away_id]]
  # probabilities = cbc.predict_proba(x)[0]
  # for i in range(len(probabilities)):
  #   print(f"{target_dict[i]}: {probabilities[i]}")

Предсказание результатов будущего матча

In [ ]:
predict_match('fc-bologna', 'fc-empoli')

1025
